# 第5章 数据类构建器

- collections.namedtuple
- typing.NamedTuple
- @dataclasses.dataclass


## 数据类构建器概述

举一个简单的类示例

直接通过类构建

In [3]:
class Coordinate: 
    def __init__(self, lat, lon): 
        self.lat = lat 
        self.lon = lon

moscow = Coordinate(55.76, 37.62)
location = Coordinate(55.76, 37.62)
moscow,location


(<__main__.Coordinate at 0x17d00dc7e80>,
 <__main__.Coordinate at 0x17d00dc7b20>)

In [4]:
location == moscow

False

In [5]:
(location.lat, location.lon) == (moscow.lat, moscow.lon)

True

In [6]:
from collections import namedtuple
Coordinate = namedtuple('Coordinate', 'lat lon')
issubclass(Coordinate, tuple)

True

In [7]:
moscow = Coordinate(55.756, 37.617)
moscow

Coordinate(lat=55.756, lon=37.617)

In [8]:
moscow == Coordinate(lat=55.756, lon=37.617)

True

In [9]:
import typing
Coordinate = typing.NamedTuple('Coordinate',
                               [('lat', float), ('lon', float)])

issubclass(Coordinate, tuple)

True

In [10]:
typing.get_type_hints(Coordinate)

{'lat': float, 'lon': float}

In [11]:
Coordinate = typing.NamedTuple('Coordinate', lat=float, lon=float)

In [12]:
moscow = Coordinate(55.756, 37.617)

In [13]:
moscow == Coordinate(lat=55.756, lon=37.617)

True

使用typing.NamedTuple构建

In [14]:
from typing import NamedTuple

class Coordinate(NamedTuple):
    lat: float
    lon: float

    def __str__(self):
        ns = 'N' if self.lat >= 0 else 'S'
        we = 'E' if self.lon >= 0 else 'W'
        return f'{abs(self.lat):.1f}°{ns}, {abs(self.lon):.1f}°{we}'


使用dataclass装饰器声明实例属性

In [15]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Coordinate:
    lat: float
    lon: float

    def __str__(self):
        ns = 'N' if self.lat >= 0 else 'S'
        we = 'E' if self.lon >= 0 else 'W'
        return f'{abs(self.lat):.1f}°{ns}, {abs(self.lon):.1f}°{we}'


## 典型的具名元组

定义并使用一个具名元组类型

In [16]:
from collections import namedtuple

City = namedtuple('City', 'name country population coordinates')
tokyo = City('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
tokyo


City(name='Tokyo', country='JP', population=36.933, coordinates=(35.689722, 139.691667))

In [17]:
tokyo.population, tokyo.coordinates, tokyo[1]

(36.933, (35.689722, 139.691667), 'JP')

具名元组的属性和方法

In [19]:
City._fields


('name', 'country', 'population', 'coordinates')

In [20]:
# 复杂一点的例子
Coordinate = namedtuple('Coordinate', 'lat lon')
delhi_data = ('Delhi NCR', 'IN', 21.935, Coordinate(28.613889, 77.208889))
delhi = City._make(delhi_data)
delhi._asdict()


{'name': 'Delhi NCR',
 'country': 'IN',
 'population': 21.935,
 'coordinates': Coordinate(lat=28.613889, lon=77.208889)}

In [21]:
delhi

City(name='Delhi NCR', country='IN', population=21.935, coordinates=Coordinate(lat=28.613889, lon=77.208889))

In [22]:
import json

json.dumps(delhi._asdict())


'{"name": "Delhi NCR", "country": "IN", "population": 21.935, "coordinates": [28.613889, 77.208889]}'

构建一个具名元组，为字段指定默认值

In [23]:
Coordinate = namedtuple('Coordinate', 'lat lon reference', defaults=['WSG84']) 
Coordinate(0, 0)


Coordinate(lat=0, lon=0, reference='WSG84')

In [24]:
Coordinate._field_defaults

{'reference': 'WSG84'}

## @dataclass 详解

定义：

```
@dataclass(*, init=True, repr=True, eq=True, order=False, unsafe_hash=False, frozen=False)
```
- init：默认值是True，生成__init__，如果用户自己实现了__init__，则忽略该参数。- 
repr：默认值是True，生成__repr__，如果用户自己实现了__repr__，则忽略该参数。- 
eq：默认值是True，生成__eq__，如果用户自己实现了__eq__，则忽略该参数- 。
order：默认值是False，生成__lt__、__le__、__gt__、__ge__、如果eq=False，或者执行定义或继承其他用于比较的方法，则抛出异- 常。
unsafe_hase：默认值是False，生成__hash- __。
frozen：默认值是False，让实例不可变。

In [27]:
# 定义ClubMember
from dataclasses import dataclass, field

@dataclass
class ClubMember:
    name: str
    guests: list = field(default_factory=list)    


都柏林核心模式

都柏林核心模式是一小组术语，可用于描述数字资源（视频、图像、网页等），也可用于描述物理资源，例如图书、CD和艺术品等对象。

In [28]:
from dataclasses import dataclass, field, fields
from typing import Optional
from enum import Enum, auto
from datetime import date


class ResourceType(Enum):  # <1>
    BOOK = auto()
    EBOOK = auto()
    VIDEO = auto()


@dataclass
class Resource:
    """描述媒体资源"""
    identifier: str                                   
    title: str = '<untitled>'                          
    creators: list[str] = field(default_factory=list)
    date: Optional[date] = None                        
    type: ResourceType = ResourceType.BOOK             
    description: str = ''
    language: str = ''
    subjects: list[str] = field(default_factory=list)
        
    def __repr__(self):
        cls = self.__class__
        cls_name = cls.__name__
        indent = ' ' * 4
        res = [f'{cls_name}(']                            
        for f in fields(cls):                             
            value = getattr(self, f.name)                 
            res.append(f'{indent}{f.name} = {value!r},')  

        res.append(')')                                   
        return '\n'.join(res)                             


In [29]:
description = 'Improving the design of existing code'
book = Resource('978-0-13-475759-9', 'Refactoring, 2nd Edition', 
                ['Martin Fowler', 'Kent Beck'], date(2018, 11, 19),
                ResourceType.BOOK, description, 'EN',
                ['computer programming', 'OOP'])


In [30]:
book


Resource(
    identifier = '978-0-13-475759-9',
    title = 'Refactoring, 2nd Edition',
    creators = ['Martin Fowler', 'Kent Beck'],
    date = datetime.date(2018, 11, 19),
    type = <ResourceType.BOOK: 1>,
    description = 'Improving the design of existing code',
    language = 'EN',
    subjects = ['computer programming', 'OOP'],
)

- Python一直都有声明类属性的简便方式，实例属性更常用，直接在__init__方法中创建实例属性。
- @dataclass解决了上述问题，类型提示始终是可选的。


## 模式匹配类实例

类模式有 3 种变体：简单类模式、关键字类模式和位置类模式。下面按 顺序依次研究。

1. 简单类模式

```
 case [str(name), _, _, (float(lat), float(lon))]:
```

- 第一项必须时str实例
- 最后一项必须时二元组，两项均为float实例
  

2. 关键字类模式

In [ ]:
import typing 
class City(typing.NamedTuple): 
    continent: str 
    name: str 
    country: str

cities = [ 
    City('Asia', 'Tokyo', 'JP'), 
    City('Asia', 'Delhi', 'IN'), 
    City('North America', 'Mexico City', 'MX'),
    City('North America', 'New York', 'US'), 
    City('South America', 'São Paulo', 'BR'),
]

def match_asian_cities(): 
    results = [] 
    for city in cities: 
        match city: 
            case City(continent='Asia'): 
                results.append(city) 
        return results

3. 位置类模式